## Sentdex Rnn with Keras
### This is an ipynb notebook that attempts to speed up and simply the creation of sequence data for the RNN that Harrison create in :
#### See (https://pythonprogramming.net/crypto-rnn-model-deep-learning-python-tensorflow-keras/)

### Imports

In [ ]:
import pandas as pd
import numpy as np
import os,sys
from collections import deque
from sklearn import preprocessing
import datetime,time

___
### Harrison's original parameters:
___

In [ ]:
# change these values
# Chose as value for SCALING_TO_USE among ['actual', 'pct', or 'minmax']
SCALING_TO_USE = 'minmax' 
SEQ_LEN = 60  # how long of a preceeding sequence to collect for RNN
FUTURE_PERIOD_PREDICT = 3  # how far into the future are we trying to predict?
RATIO_TO_PREDICT = "LTCUSD" # change this to use different currencies as the target
EPOCHS = 3  # how many passes through our data
BATCH_SIZE = 64  # how many batches? Try smaller batch if you're getting OOM (out of memory) errors.

# DO NOT CHANGE THE VALUES BELOW
DATA_FOLDER = './data/crypto_data'
CURRENCY_FILE_NAMES = os.listdir(DATA_FOLDER) 
CURRENCY_LIST = [c.replace('-','').replace('.csv','')for c in CURRENCY_FILE_NAMES]
if RATIO_TO_PREDICT not in CURRENCY_LIST:
    raise ValueError(f'the currency {RATIO_TO_PREDICT} is not in the valid currency list {CURRENCY_LIST}')
print(f'possible currencies to use below are: {[c.replace("-","").replace(".csv","") for c in CURRENCY_FILE_NAMES]}')


NAME = f"{SEQ_LEN}-SEQ-{FUTURE_PERIOD_PREDICT}-PRED-{int(time.time())}"

print(f'using {SCALING_TO_USE} as scaling')
print(f'using {RATIO_TO_PREDICT} currency as a target')

___
### Create DataFrame of close and volume for each currency
___

In [ ]:
days_in_future = FUTURE_PERIOD_PREDICT

df_curr = None
for i in range(len(CURRENCY_LIST)):
    cfn = CURRENCY_FILE_NAMES[i]
    c  = CURRENCY_LIST[i]
    df_temp = pd.read_csv(f'{DATA_FOLDER}/{cfn}')
    df_temp = df_temp.rename(columns={'close':f'close_{c}','volume':f'volume_{c}'})
    df_temp.index = df_temp.timestamp
    df_temp = df_temp[[f'close_{c}',f'volume_{c}']]
    if i ==0:
        df_curr = df_temp.copy()
    else:
        df_curr = pd.concat([df_curr,df_temp],axis=1)
close_string = f'close_{RATIO_TO_PREDICT}'
future_close_string = f'close_{RATIO_TO_PREDICT}_future'
df_curr[future_close_string] = df_curr[close_string].shift(-days_in_future)
df_curr['target'] = df_curr[future_close_string] >= df_curr[close_string]
df_curr['hour'] = [datetime.datetime.fromtimestamp(d).hour for d  in df_curr.index]
df_curr['minute'] = [datetime.datetime.fromtimestamp(d).minute for d  in df_curr.index]
df_curr['year'] = [datetime.datetime.fromtimestamp(d).year for d  in df_curr.index]
df_curr['month'] = [datetime.datetime.fromtimestamp(d).month for d  in df_curr.index]
df_curr['day'] = [datetime.datetime.fromtimestamp(d).day for d  in df_curr.index]

# get all columns except future_close_string
cols = list(filter(lambda co: co!=future_close_string,df_curr.columns.values))


df_curr = df_curr[(df_curr.hour>=8) & (df_curr.hour<=17)]
df_curr['yyyymmdd'] = df_curr.year*100*100 + df_curr.month*100 + df_curr.day
df_curr = df_curr[['yyyymmdd','hour','minute']+cols]

len(df_curr),df_curr.columns.values


___
### Create separate training and validation DataFrames
___

In [ ]:
# get all yyyymmdd's  in df_curr.yyyymmdd
all_days = np.array(list(set(df_curr.yyyymmdd.as_matrix().reshape(-1))))
# establish a training size percent
train_perc = .9
# get the index into the array all_days, which represents the last training day
last_train_index = int(len(all_days)*train_perc)
# get training days
train_days = all_days[:last_train_index]
# get df_train
df_train = df_curr[df_curr.yyyymmdd.isin(train_days)]
# get test days
test_days = all_days[last_train_index:]
# get df_test
df_test = df_curr[df_curr.yyyymmdd.isin(test_days)]
# print various lengths to make sure the df_train and df_test DataFrames sum to df_curr
print(f'df_curr:{len(df_curr)}, df_train:{len(df_train)}, df_test:{len(df_test)}, should be 0: {len(df_curr)-(len(df_train)+len(df_test))}')


___
### normalize data
For both training and validation data, provide return a dictionary with 3 DataFrames:
1. A DataFrame of actual prices, and the actual target values
2. A DataFrame with percentage prices, and the actual target values
3. A DataFrame with min/max normalized percentage of the prices, and the actual (not normalized) target values.  
___

In [ ]:
def normalize_df(df):
    num_cols = list(filter(lambda c:'close' in c or 'volume' in c,df_train.columns.values))
    dft = df.copy() #[['yyyymmdd','hour','minute']+num_cols]
    dft.target = dft.target.astype(int)
    dft = dft[~dft.isnull().any(axis=1)]
    dft_actual = dft.copy()
    dft[num_cols] = dft[num_cols].pct_change()
    dft = dft[~dft.isnull().any(axis=1)]
    dft_pct = dft.copy()
    dft[num_cols] = (dft[num_cols] - dft[num_cols].min()) / (dft[num_cols].max() - dft[num_cols].min())
    dft_minmax = dft.copy()
    return {'actual':dft_actual,'pct':dft_pct,'minmax':dft_minmax}

training = normalize_df(df_train)
testing = normalize_df(df_test)

In [ ]:
training['actual'].head()

In [ ]:
training['pct'].head()

In [ ]:
training['minmax'].head()

___
###  Create sequences from the minmax DataFrames
    From the input DataFrame df, create 2 numpy arrays:
    1. A numpy array of "time sequences"
    2. A numpy array of target values, that coincide with each sequence

___

In [ ]:

def create_sequences(df):
    '''
    From the input DataFrame df, a new DataFrame with::
    1. an 'x' column where each cell is an array of "time sequences" (shape of array in cell = (SEQ_LEN,num_x_cols))
    2. a 'y' column where each cell is an array of "time sequences" (shape of array in cell = (SEQ_LEN,1))
    '''
    
    num_cols = list(filter(lambda c: 'close' in c or 'volume' in c,df.columns.values))
    df2 = df.copy()
    df2['ts'] = df2.index
    df2.index = list(range(len(df2)))
    dfy = df2[['target']]
    dfx = df2.drop(['target'],axis=1)
    dx = dfx.as_matrix()
    
    dy = dfy.as_matrix()
    s = len(dx)
    sl = 60
    last_col = len(df2.columns.values) -1
    x_sequences = [dx[i:None if i+sl>=s else i+sl] for i in range(len(dx)-(sl-1))]
    y_sequences = [dy[i:None if i+sl>=s else i+sl] for i in range(len(dy)-(sl-1))]
    x_seq_dicts = [{'x':x_sequences[i]} for i in list(range(len(x_sequences)))]
    y_seq_dicts = [{'y':y_sequences[i]} for i in list(range(len(y_sequences)))]
    dfx = pd.DataFrame(x_seq_dicts)
    dfy = pd.DataFrame(y_seq_dicts)
    df_seq = dfx.join(dfy)
    return df_seq


from tqdm import tqdm
def create_daily_sequences(df):
    '''
    Given and input DataFrame df, run create_sequences on subsets of df 
       where each subset contains data for only a specific day.
    Combine all of the specific "daily" calls to create_sequences into a single x and y array.
    '''
    cols_to_return = list(filter(lambda c: any([d in c for d in ['close','volume','target']]),df.columns.values))
    df_seqs = None
    all_days = np.array(list(set(df.yyyymmdd)))
    for yyyymmdd in tqdm(all_days):
        df_this_day = df[df.yyyymmdd==yyyymmdd][cols_to_return]
        df_this_seqs = create_sequences(df_this_day)
        if df_seqs is None:
            df_seqs = df_this_seqs.copy()
        else:
            if len(df_this_seqs)>0:
                df_seqs = df_seqs.append(df_this_seqs)
    return df_seqs


#### Run create_daily_sequences to get final input data for the RNN

In [ ]:
df_train_seqs = create_daily_sequences(training[SCALING_TO_USE])
train_x = np.array([df_train_seqs['x'].values[i] for i in range(len(df_train_seqs))])
train_y = np.array([df_train_seqs['y'].values[i] for i in range(len(df_train_seqs))])

df_test_seqs = create_daily_sequences(testing[SCALING_TO_USE])
validation_x = np.array([df_test_seqs['x'].values[i] for i in range(len(df_test_seqs))])
validation_y = np.array([df_test_seqs['y'].values[i] for i in range(len(df_test_seqs))])

txl = train_x.shape[0]//BATCH_SIZE * BATCH_SIZE
train_x_even = train_x[:txl]
train_y_even = train_y[:txl]

vxl = validation_x.shape[0]//BATCH_SIZE * BATCH_SIZE
validation_x_even =  validation_x[:vxl]
validation_y_even =  validation_y[:vxl]

train_x_even.shape,train_y_even.shape,validation_x_even.shape,validation_y_even.shape


___
# TO DO, implement this
### Balance data
Create sequences whose corrosponding target values have an equal number of 0's and 1's
___

In [ ]:
def balance_data(tseqs):
    # get the target (y) values from the tsegs['y'] array, and put it into a DataFrame for indexing
    dfy  = pd.DataFrame({'y':tseqs['y'].reshape(-1)})
    # split dfy into 2 DataFrames: dfy0 contains 0's, and dfy1 contains 1's
    dfy0 = dfy[dfy.y==0]
    dfy1 = dfy[dfy.y==1]
    # find the smaller DataFrame
    dfy_len = min(len(dfy0),len(dfy1))
    # create a new DataFrame which contains an equal number of targets with 1's and 0's
    dfy01 = dfy0.iloc[:dfy_len].append(dfy1.iloc[:dfy_len])
    # get the index values of this new DataFrame
    dfy_indices= dfy01.index
    # use those index values to locate the x sequences that corrospond to the targets in dfy01
    tsy = tseqs['y'][dfy_indices]
    tsx = tseqs['x'][dfy_indices]
    # return 2 arrays: 1 for the x sequences and another for the corrosponding targets
    return tsx,tsy



#### Run balance_data

___
## Now create the Keras model and run it
___

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RNN, Dense, Dropout, LSTM, CuDNNLSTM, BatchNormalization,Flatten,SimpleRNN
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import ModelCheckpoint
import time


model = Sequential()
# model.add(LSTM(128, input_shape=(train_x.shape[1:]), return_sequences=True))
model.add(LSTM(128, return_sequences=True,stateful=True,
              batch_input_shape=(BATCH_SIZE,train_x.shape[1],train_x.shape[2])))
# model.add(SimpleRNN(128))
model.add(Dropout(0.2))
model.add(BatchNormalization())  #normalizes activation outputs, same reason you want to normalize your input data.

# # model.add(LSTM(128, return_sequences=True))
# model.add(LSTM(128, return_sequences=True,stateful=True,
#               batch_input_shape=(BATCH_SIZE,train_x.shape[1],train_x.shape[2])))
# model.add(Dropout(0.1))
# model.add(BatchNormalization())

# # model.add(LSTM(128))
# model.add(LSTM(128, return_sequences=True,stateful=True,
#               batch_input_shape=(BATCH_SIZE,train_x.shape[1],train_x.shape[2])))
# model.add(Dropout(0.2))
# model.add(BatchNormalization())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(2, activation='softmax'))


In [ ]:
opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=opt,
    metrics=['accuracy']
)

# I commented out the tensorboard stuff
# tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

# filepath = "RNN_Final-{epoch:02d}-{val_acc:.3f}"  # unique file name that will include the epoch and the validation acc for that epoch
# checkpoint = ModelCheckpoint("models/{}.model".format(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')) # saves only the best ones

# Train model
history = model.fit(
    train_x_even, train_y_even,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_x_even, validation_y_even),
#     callbacks=[tensorboard, checkpoint],
)
